<a href="https://colab.research.google.com/github/1213kush/1213kush/blob/main/clinetex1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
!pip install opencv-python

In [11]:
!pip install pillow

In [12]:
!pip install scikit-image

In [13]:
from PIL import Image
import os
from google.colab import drive

# Mount Google Drive if necessary
drive.mount('/content/drive')

dataset_path = '/content/drive/MyDrive/ clintex_dataset/datasetkim'

# Print the contents of the folder for debugging
print("Files in the folder:")
for filename in os.listdir(dataset_path):
    print(filename)

# Iterate through files and display images
for filename in os.listdir(dataset_path):
    if filename.endswith(('.jpg', '.jpeg', '.png')):
        image_path = os.path.join(dataset_path, filename)
        try:
            image = Image.open(image_path)
            print(f"Displaying: {filename}") # Print the filename before displaying
            image.show()
            break  # Stop after displaying the first image or remove to show all
        except Exception as e:
            print(f"Error opening {filename}: {e}") # Print errors during image opening

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Files in the folder:
Benzoistrol-1.png
(a) Oxytetracyclline.png
Chlor Promazine-3.png
Dextroraphan.png
Antivenom-5.png
Carbacoal-1.png
Dhatura-4.png
Butyl cloral Hydrate-4.png
Acepromazin Maliate-4.png
Antimani Potasium Tartrate.png
Dihydro Codeinone-1.png
Digitalics and Its Glycosides-2.png
Bitameprodine.png
Chloromethene Hydrochloride-3.png
Diazepam.png
Antigen-1.png
Azapentine-5.png
Apomorphine.png
Dhatura-1.png
Allopurinol-2.png
Barbital-4.png
Antigen-4.png
Allopurinol-4.png
6 – Mercaptopurine-5.png
Codeine-1.png
Azathioprine-2.png
Acetyl methadol-1.png
Codeine Methyl Bromide-5.png
Dihydro Codeine-4.png
Azathioprine-1.png
Bendrofluazide-3.png
Clonitazine.png
(c) Procane Benzyl Penicillin-2.png
Dapsone-3.png
Chlorambucil-2.png
Captodine.png
Diethyl thyambutine-4.png
Anti Rabies Serum-4.png
Chloroform-5.png
Desomorphine-1.png
Antigen-2.png
Dimercoprol-1.png

In [14]:
!pip install pytesseract openpyxl

In [15]:
import pytesseract
from PIL import Image
import os
import pandas as pd
import re


In [16]:
# Path where images are stored in Google Drive
folder_path = '/content/drive/MyDrive/ clintex_dataset/datasetkim'


In [17]:
def extract_entities(text):
    name = None
    date = None
    doctor_name = None
    medicines = []

    # Extract date (formats like 12-03-2024 or 12/03/2024)
    date_match = re.search(r'(\d{2}[-/]\d{2}[-/]\d{4})', text)
    if date_match:
        date = date_match.group(1)

    # Extract Doctor Name (if line starts with Dr. or Doctor)
    doctor_match = re.search(r'(Dr\.?\s+[A-Za-z ]+)', text)
    if doctor_match:
        doctor_name = doctor_match.group(1).strip()

    # Extract Patient Name (simple assumption, line starting with Name or Patient Name)
    name_match = re.search(r'(?:Patient Name|Name)\s*[:\-]?\s*(.+)', text, re.IGNORECASE)
    if name_match:
        name = name_match.group(1).strip()

    # Extract medicines (lines containing mg or ml)
    medicine_lines = []
    for line in text.split('\n'):
        line = line.strip()
        if len(line) > 0 and re.search(r'\bmg\b|\bml\b', line.lower()):
            medicine_lines.append(line)
    medicines = '; '.join(medicine_lines)

    return name, date, doctor_name, medicines


In [18]:
# List to collect structured data
structured_data = []

# Loop through images
for filename in os.listdir(folder_path):
    if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
        img_path = os.path.join(folder_path, filename)
        img = Image.open(img_path)

        # OCR
        text = pytesseract.image_to_string(img)

        # Extract entities
        name, date, doctor_name, medicines = extract_entities(text)

        # Store into list
        structured_data.append({
            'filename': filename,
            'patient_name': name,
            'date': date,
            'doctor_name': doctor_name,
            'medicines': medicines,
            'full_extracted_text': text  # keep full text also if needed
        })


In [19]:
import re
from openpyxl.utils import escape

# Function to clean text
def clean_text(text):
    if text is not None:
        # Remove illegal unicode characters (like \x0c form feed)
        text = re.sub(r'[\x00-\x08\x0b\x0c\x0e-\x1f]', '', text)
    return text

# Apply cleaning to all text fields
for item in structured_data:
    item['patient_name'] = clean_text(item['patient_name'])
    item['doctor_name'] = clean_text(item['doctor_name'])
    item['medicines'] = clean_text(item['medicines'])

    item['full_extracted_text'] = clean_text(item['full_extracted_text'])


In [20]:
# Now safely create DataFrame and save
df = pd.DataFrame(structured_data)

output_path = '/content/drive/MyDrive/clintex_dataset/structured_output.xlsx'
df.to_excel(output_path, index=False)

print(f"✅ Successfully saved cleaned structured data to {output_path}")


✅ Successfully saved cleaned structured data to /content/drive/MyDrive/clintex_dataset/structured_output.xlsx


In [21]:
!mkdir -p /content/drive/MyDrive/clintex_dataset/dataset


In [22]:
!cp /content/A_Z_medicines_dataset_of_India.csv /content/drive/MyDrive/clintex_dataset/dataset

cp: cannot stat '/content/A_Z_medicines_dataset_of_India.csv': No such file or directory


In [23]:
import pandas as pd

df =pd.read_csv('/content/drive/MyDrive/clintex_dataset/dataset/A_Z_medicines_dataset_of_India.csv')

print(df.head(5))

   id                      name  price(₹)  Is_discontinued  \
0   1  Augmentin 625 Duo Tablet    223.42            False   
1   2       Azithral 500 Tablet    132.36            False   
2   3          Ascoril LS Syrup    118.00            False   
3   4      Allegra 120mg Tablet    218.81            False   
4   5            Avil 25 Tablet     10.96            False   

                      manufacturer_name       type         pack_size_label  \
0  Glaxo SmithKline Pharmaceuticals Ltd  allopathy     strip of 10 tablets   
1           Alembic Pharmaceuticals Ltd  allopathy      strip of 5 tablets   
2          Glenmark Pharmaceuticals Ltd  allopathy  bottle of 100 ml Syrup   
3                     Sanofi India  Ltd  allopathy     strip of 10 tablets   
4                     Sanofi India  Ltd  allopathy     strip of 15 tablets   

      short_composition1          short_composition2  
0  Amoxycillin  (500mg)      Clavulanic Acid (125mg)  
1   Azithromycin (500mg)                        

In [24]:
print(df.tail(5))

            id                           name  price(₹)  Is_discontinued  \
253968  253969  Ziyapod 100mg Oral Suspension      62.3            False   
253969  253970            Zemhart 30mg Tablet      54.0            False   
253970  253971              Zivex 25mg Tablet      57.0            False   
253971  253972        ZI Fast 500mg Injection     152.0            False   
253972  253973      Zyvocol 1% Dusting Powder     110.0            False   

                            manufacturer_name       type  \
253968            Ziyana Lifesciences Pvt Ltd  allopathy   
253969                 Leeford Healthcare Ltd  allopathy   
253970                          Euro Organics  allopathy   
253971  Burgeon Health Series Private Limited  allopathy   
253972                         GBK Healthcare  allopathy   

                        pack_size_label            short_composition1  \
253968  bottle of 30 ml Oral Suspension  Cefpodoxime Proxetil (100mg)   
253969              strip of 10 tabl

In [25]:
print(df.isnull().sum)

<bound method DataFrame.sum of            id   name  price(₹)  Is_discontinued  manufacturer_name   type  \
0       False  False     False            False              False  False   
1       False  False     False            False              False  False   
2       False  False     False            False              False  False   
3       False  False     False            False              False  False   
4       False  False     False            False              False  False   
...       ...    ...       ...              ...                ...    ...   
253968  False  False     False            False              False  False   
253969  False  False     False            False              False  False   
253970  False  False     False            False              False  False   
253971  False  False     False            False              False  False   
253972  False  False     False            False              False  False   

        pack_size_label  short_composition1 

In [26]:
print((df.isnull().sum() /len(df))*100)

id                     0.000000
name                   0.000000
price(₹)               0.000000
Is_discontinued        0.000000
manufacturer_name      0.000000
type                   0.000000
pack_size_label        0.000000
short_composition1     0.000000
short_composition2    55.833494
dtype: float64


In [27]:
# Show all rows that have at least one missing value
df[df.isnull().any(axis=1)]


,id,name,price(₹),Is_discontinued,manufacturer_name,type,pack_size_label,short_composition1,short_composition2
1,2,Azithral 500 Tablet,132.36,False,Alembic Pharmaceuticals Ltd,allopathy,strip of 5 tablets,Azithromycin (500mg),NaN
3,4,Allegra 120mg Tablet,218.81,False,Sanofi India Ltd,allopathy,strip of 10 tablets,Fexofenadine (120mg),NaN
4,5,Avil 25 Tablet,10.96,False,Sanofi India Ltd,allopathy,strip of 15 tablets,Pheniramine (25mg),NaN
7,8,Azee 500 Tablet,132.38,False,Cipla Ltd,allopathy,strip of 5 tablets,Azithromycin (500mg),NaN
8,9,Atarax 25mg Tablet,85.50,False,Dr Reddy's Laboratories Ltd,allopathy,strip of 15 tablets,Hydroxyzine (25mg),NaN
...,...,...,...,...,...,...,...,...,...
253968,253969,Ziyapod 100mg Oral Suspension,62.30,False,Ziyana Lifesciences Pvt Ltd,allopathy,bottle of 30 ml Oral Suspension,Cefpodoxime Proxetil (100mg),NaN
253969,253970,Zemhart 30mg Tablet,54.00,False,Leeford Healthcare Ltd,allopathy,strip of 10 tablets,Diltiazem (30mg),NaN
253970,253971,Zivex 25mg Tablet,57.00,False,Euro Organics,allopathy,strip of 10 tablets,Hydroxyzine (25mg),NaN
253971,253972,ZI Fast 500mg Injection,152.00,False,Burgeon Health Series Private Limited,allopathy,vial of 1 Injection,Azithromycin (500mg),NaN


In [28]:
df.drop(['short_composition2'], axis =1, inplace = True)

In [29]:
print((df.isnull().sum()) / len(df)*100)

id                    0.0
name                  0.0
price(₹)              0.0
Is_discontinued       0.0
manufacturer_name     0.0
type                  0.0
pack_size_label       0.0
short_composition1    0.0
dtype: float64


In [30]:
print(df.isnull().any(axis=1))

0         False
1         False
2         False
3         False
4         False
          ...  
253968    False
253969    False
253970    False
253971    False
253972    False
Length: 253973, dtype: bool


In [31]:
# Show all rows that have at least one missing value
df[df.isnull().any(axis=1)]

,id,name,price(₹),Is_discontinued,manufacturer_name,type,pack_size_label,short_composition1


In [32]:
print(df.columns)

Index(['id', 'name', 'price(₹)', 'Is_discontinued', 'manufacturer_name',
       'type', 'pack_size_label', 'short_composition1'],
      dtype='object')


In [33]:
print(df.head(5).any(axis=1))

0    True
1    True
2    True
3    True
4    True
dtype: bool


In [34]:
import pandas as pd
from PIL import Image
import pytesseract
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import difflib
import re

In [35]:
import pandas as pd
import re

# Step 1: Load your dataset
df = pd.read_csv('/content/drive/MyDrive/clintex_dataset/dataset/A_Z_medicines_dataset_of_India.csv')

# Step 2: Lowercase important text columns
df['medicine_name'] = df['name'].str.lower()
df['short_composition1'] = df['short_composition1'].str.lower()
df['pack_size_label'] = df['pack_size_label'].astype(str).str.lower()
df['manufacturer_name'] = df['manufacturer_name'].astype(str).str.lower()
df['type'] = df['type'].astype(str).str.lower()

# If 'price(竄ｹ)' has wrong characters, first fix its name
if 'price(竄ｹ)' in df.columns:
    df.rename(columns={'price(竄ｹ)': 'price(₹)'}, inplace=True)

# Step 3: Remove duplicates (based on all important columns)
df = df.drop_duplicates(subset=['medicine_name', 'short_composition1', 'pack_size_label', 'manufacturer_name', 'type'])

# Step 4: Clean unwanted special characters
def clean_text(text):
    if isinstance(text, str):
        text = re.sub(r'[^a-zA-Z0-9\s\.-₹]', '', text)  # Allow ₹ symbol too
        text = re.sub(r'\s+', ' ', text).strip()
    return text

for col in ['medicine_name', 'short_composition1', 'pack_size_label', 'manufacturer_name', 'type']:
    df[col] = df[col].apply(clean_text)

# Step 5: Normalize dosages inside 'short_composition1'
def normalize_dosage(dosage):
    if isinstance(dosage, str):
        dosage = dosage.lower()
        dosage = dosage.replace('mg', ' mg')
        dosage = dosage.replace('ml', ' ml')
        dosage = dosage.replace('mcg', ' mcg')
        dosage = re.sub(r'\s+', ' ', dosage).strip()
    return dosage

df['short_composition1'] = df['short_composition1'].apply(normalize_dosage)

# Step 6: Save the cleaned expanded dataset
output_path = '/content/drive/MyDrive/clintex_dataset/cleaned_expanded_drug_dataset.csv'

# 🔥 Now including price(₹) also in output
df[['medicine_name', 'short_composition1', 'pack_size_label', 'manufacturer_name', 'type', 'price(₹)']].to_csv(output_path, index=False)

print(f"✅ Cleaned Expanded Dataset saved at {output_path}")


✅ Cleaned Expanded Dataset saved at /content/drive/MyDrive/clintex_dataset/cleaned_expanded_drug_dataset.csv


In [36]:
!pip install easyocr
!pip install pytesseract
!apt install tesseract-ocr
!pip install fuzzywuzzy
!pip install rapidfuzz
!pip install python-Levenshtein


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 34 not upgraded.


In [37]:
import easyocr
import pytesseract
import cv2
import matplotlib.pyplot as plt
from fuzzywuzzy import process
import pandas as pd
import numpy as np
import re
import difflib


In [38]:
# Load cleaned dataset
df = pd.read_csv('/content/drive/MyDrive/clintex_dataset/cleaned_expanded_drug_dataset.csv')


In [39]:
reader = easyocr.Reader(['en'])

def extract_text_from_image(image_path):
    # First try easyocr
    result = reader.readtext(image_path, detail=0)
    extracted_text = ' '.join(result)

    # Backup using pytesseract
    try:
        backup_text = pytesseract.image_to_string(image_path)
        combined = extracted_text + " " + backup_text
    except:
        combined = extracted_text

    combined = combined.lower()
    combined = re.sub(r'[^a-zA-Z0-9\s]', '', combined)  # Clean special characters
    combined = re.sub(r'\s+', ' ', combined).strip()
    return combined


In [40]:
def suggest_medicines(extracted_text, top_n=5):
    medicines = df['medicine_name'].tolist()

    # Use fuzzy matching
    matches = process.extract(extracted_text, medicines, limit=top_n)

    suggestions = []
    for match in matches:
        med_name = match[0]
        row = df[df['medicine_name'] == med_name].iloc[0]

        suggestion = {
            'medicine_name': row['medicine_name'],
            'short_composition1': row['short_composition1'],
            'pack_size_label': row['pack_size_label'],
            'manufacturer_name': row['manufacturer_name'],
            'type': row['type'],
            'price(₹)': row['price(₹)']
        }
        suggestions.append(suggestion)

    return suggestions


In [44]:
# Example usage:

# 1. Upload a prescription image
image_path = '/content/124.jpg'  # Update your path

# 2. Extract text
extracted_text = extract_text_from_image(image_path)
print("📝 Extracted Text from Prescription:")
print(extracted_text)

# 3. Suggest medicines
suggestions = suggest_medicines(extracted_text)

# 4. Show suggestions
print("\n💊 Medicine Suggestions:")
for idx, med in enumerate(suggestions, 1):
    print(f"\n🔹 Suggestion {idx}")
    for k, v in med.items():
        print(f"{k}: {v}")


📝 Extracted Text from Prescription:
netaji subhas chandra bose cancer hospital palaccreditto of himadri memorial cancer welfare trust 6 deak 3081 nayabad kolkau 700 094india vms 038 7122300 mob 9874210003 9874210007 etnle8nch n admingnscn i web www nscni in doctor prescription nchit name dalia kundu no ncri211427 consultant appointment kumar andul sc height weight bp 402202 history symptoms clinical examination gs 3 eove xvw diagnosis 0a zeho medicine prescribed pb av gph i xgeva mxt 4 cc 120 cive g ph ow i0221 advice prescribed in kav ow ris ow 2 cee hmw tdc clofeutb v 2 follow up date t boralu b0 m2ay t ovdouy 0a2by taqa l tultac9 obaana hebyf kumar mandal rgg no wbvc63430 2 mdinternal medicine dmmedical oncology atrz p medieal oncologykc gondarl f cost 1 sh 8 unite ncri help paticnt namne mr tanmoy date age pulse oigci mec kum zlzl2y mandal tejy mof dm oncologz oncologist medicinok dr hcdicel hemalooncologist mdr amnal modical wbmc63430 urcp consultant no rogn netaji su bhas chandra

In [45]:
# Print column names to verify
print(medicine_db.columns)


Index(['medicine_name', 'short_composition1', 'pack_size_label',
       'manufacturer_name', 'type', 'price(₹)'],
      dtype='object')


In [47]:
# Install libraries (uncomment if needed)
# !pip install gradio easyocr pandas

import gradio as gr
import easyocr
import pandas as pd

# Load OCR reader
reader = easyocr.Reader(['en'])

# Load enhanced medicine database
medicine_db = pd.read_csv('/content/drive/MyDrive/clintex_dataset/cleaned_expanded_drug_dataset.csv')

# Clean column names (important!)
medicine_db.columns = medicine_db.columns.str.strip()

def extract_and_suggest(image):
    # Step 1: OCR extraction
    result = reader.readtext(image)
    extracted_text = " ".join([text[1] for text in result])

    # Step 2: Find matching medicines
    suggestions = []
    for med in medicine_db['medicine_name']:
        if med.split()[0].lower() in extracted_text.lower():
            alt_meds = medicine_db[medicine_db['medicine_name'].str.contains(med.split()[0], case=False)]
            alt_meds = alt_meds.sort_values('price(₹)')
            suggestions.append(alt_meds)

    # Step 3: Prepare output text
    final_output = ""

    # Extracted Text Section
    final_output += "📝 **Extracted Text from Prescription:**\n\n"
    final_output += f"> {extracted_text}\n\n"

    # Medicine Suggestions Section
    if not suggestions:
        final_output += "💊 **Medicine Suggestions:**\n\n"
        final_output += "> No medicines found. Try uploading a clearer prescription image."
    else:
        final_output += "💊 **Medicine Suggestions:**\n"
        for idx, alternatives in enumerate(suggestions, start=1):
            for _, row in alternatives.iterrows():
                final_output += f"\n🔹 **Suggestion {idx}**\n"
                final_output += (
                    f"- **Medicine Name**: {row['medicine_name']}\n"
                    f"- **Short Composition**: {row['short_composition1']}\n"
                    f"- **Pack Size**: {row['pack_size_label']}\n"
                    f"- **Manufacturer**: {row['manufacturer_name']}\n"
                    f"- **Type**: {row['type']}\n"
                    f"- **Price(₹)**: ₹{row['price(₹)']}\n"
                )
                break  # Only show the cheapest alternative for each match

    return final_output

# Gradio Interface
interface = gr.Interface(
    fn=extract_and_suggest,
    inputs=gr.Image(type="pil"),
    outputs=gr.Markdown(),
    title="Prescription OCR & Medicine Suggestions",
    description="📄 Upload your medical prescription image and get cheaper alternative medicines with composition, type, pack size & price!",
    allow_flagging="never"
)

# Launch
interface.launch(share=True)


/usr/local/lib/python3.11/dist-packages/gradio/interface.py:415: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated.Use `flagging_mode` instead.
  warnings.warn(


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://363c964c810295f466.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [50]:
# Install libraries (uncomment if needed)
# !pip install gradio easyocr pandas

import gradio as gr
import easyocr
import pandas as pd
import numpy as np

# Load OCR reader
reader = easyocr.Reader(['en'])

# Load medicine database
medicine_db = pd.read_csv('/content/drive/MyDrive/clintex_dataset/cleaned_expanded_drug_dataset.csv')

# Clean column names
medicine_db.columns = medicine_db.columns.str.strip()

def extract_and_suggest(image):
    # Convert PIL Image to numpy array
    image_np = np.array(image)

    # Step 1: OCR extraction
    result = reader.readtext(image_np)
    extracted_text = " ".join([text[1] for text in result])

    # Step 2: Find matching medicines
    suggestions = []
    for med in medicine_db['medicine_name']:
        if med.split()[0].lower() in extracted_text.lower():
            alt_meds = medicine_db[medicine_db['medicine_name'].str.contains(med.split()[0], case=False)]
            alt_meds = alt_meds.sort_values('price(₹)')
            suggestions.append(alt_meds)

    # Step 3: Prepare outputs
    extracted_output = "📝 **Extracted Text from Prescription:**\n\n"
    extracted_output += f"> {extracted_text}"

    suggestion_output = "💊 **Medicine Suggestions:**\n\n"
    if not suggestions:
        suggestion_output += "> No medicines found. Try uploading a clearer prescription image."
    else:
        for idx, alternatives in enumerate(suggestions, start=1):
            for _, row in alternatives.iterrows():
                suggestion_output += f"\n🔹 **Suggestion {idx}**\n"
                suggestion_output += (
                    f"- **Medicine Name**: {row['medicine_name']}\n"
                    f"- **Short Composition**: {row['short_composition1']}\n"
                    f"- **Pack Size**: {row['pack_size_label']}\n"
                    f"- **Manufacturer**: {row['manufacturer_name']}\n"
                    f"- **Type**: {row['type']}\n"
                    f"- **Price(₹)**: ₹{row['price(₹)']}\n"
                )
                break  # Only pick the first cheapest one

    return extracted_output, suggestion_output

# Gradio Interface
interface = gr.Interface(
    fn=extract_and_suggest,
    inputs=gr.Image(type="pil"),
    outputs=[
        gr.Markdown(label="Extracted Text"),
        gr.Markdown(label="Medicine Suggestions")
    ],
    title="🩺 Prescription OCR & Medicine Alternatives",
    description="📤 Upload your prescription image and get text extracted + cheaper alternative medicines!",
    allow_flagging="never",
    theme="default",  # Looks clean
)

# Launch
interface.launch(share=True)


/usr/local/lib/python3.11/dist-packages/gradio/interface.py:415: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated.Use `flagging_mode` instead.
  warnings.warn(


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://f6d233bfadb8dcda29.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
